In [ ]:
import pandas as pd


In [ ]:
data = pd.read_csv('职位描述_技术栈.csv')
print(data.columns)


In [ ]:
# 数据预处理步骤：

# 1. 清理岗位描述中的文本，去除无关字符。
# 2. 清理技术栈，去除换行符和空格，并分割技能。
# 3. 分割数据集为训练集、验证集和测试集。

# 清理岗位描述
data['岗位描述清洁'] = data['岗位描述'].str.replace('财务软件工程师任职要求：', '')
data['岗位描述清洁'] = data['岗位描述清洁'].str.replace('岗位职责：', '')
data['岗位描述清洁'] = data['岗位描述清洁'].str.replace('工作职责岗位职责：', '')
data['岗位描述清洁'] = data['岗位描述清洁'].str.replace('工作职责：', '')
data['岗位描述清洁'] = data['岗位描述清洁'].str.replace('任职要求：', '')
data['岗位描述清洁'] = data['岗位描述清洁'].str.strip()

# 清理技术栈，分割技能
data['技术栈清洁'] = data['技术栈'].str.replace('\n', ' ').str.split(r'\s*;\s*|\s*,\s*|\s+')

# 显示预处理后的数据
data[['岗位描述清洁', '技术栈清洁']].head()


In [ ]:
from sklearn.model_selection import train_test_split

# 由于我们无法在这个环境中加载BERT模型，我们将演示如何准备数据。

# 将数据分为岗位描述（X）和技术栈（y）
X = data['岗位描述清洁'].tolist()
y = data['技术栈清洁'].tolist()

# 分割数据集为训练集和测试集（我们可以进一步将训练集分为训练集和验证集）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 显示分割结果的大小
len(X_train), len(X_test)


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

# 加载分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

# 加载预训练的BERT模型
model = TFBertForSequenceClassification.from_pretrained('bert-base-chinese')


In [ ]:
# 定义一个函数来将文本转换为模型所需的格式
def convert_examples_to_features(examples, labels, max_seq_length, tokenizer):
    input_ids, attention_masks, token_type_ids, label_ids = [], [], [], []

    for example, label in zip(examples, labels):
        # 使用分词器标记化文本
        bert_input = tokenizer.encode_plus(
            example, 
            add_special_tokens=True, 
            max_length=max_seq_length, 
            pad_to_max_length=True,
            return_attention_mask=True
        )
        
        input_ids.append(bert_input['input_ids'])
        attention_masks.append(bert_input['attention_mask'])
        token_type_ids.append(bert_input['token_type_ids'])
        label_ids.append(label)

    return (np.array(input_ids), np.array(attention_masks), np.array(token_type_ids)), np.array(label_ids)

# 转换数据
max_seq_length = 128
train_features, train_labels = convert_examples_to_features(X_train, y_train, max_seq_length, tokenizer)
test_features, test_labels = convert_examples_to_features(X_test, y_test, max_seq_length, tokenizer)


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy

# 编译模型
optimizer = Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = BinaryCrossentropy(from_logits=True)
metric = BinaryAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# 训练模型
model.fit(
    train_features,
    train_labels,
    validation_split=0.1,
    epochs=3,
    batch_size=32
)


In [ ]:
# 评估模型
model.evaluate(test_features, test_labels)

# 进行预测
predictions = model.predict(test_features)
